In [13]:
import numpy as np
from sklearn.preprocessing import StandardScaler


In [2]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

def sigmoid_derivative(X):
    return X* (1-X)

def relu(X):
    return np.maximum(0, X)

def relu_derivative(X):
    return np.where(X > 0, 1, 0)


def crossentropyloss(y, y_preds):
        y_preds = np.clip(y_preds, 1e-15, 1 - 1e-15)
        loss = -np.mean( y * np.log(y_preds) + (1-y) * np.log(1-y_preds))
        return loss

class NeuralNetwork():
    def __init__(self, input_size: int, hidden_units: int, output_size:int):
        # self.weights_ip = np.round(np.random.rand(input_size, hidden_units),2)  
        # self.bias_ip = np.zeros((hidden_units)) 
        # self.weights_op = np.round(np.random.rand(hidden_units, output_size),2)  
        # self.bias_op = np.zeros((output_size)) 
        self.weights_ip = np.random.randn(input_size, hidden_units) * np.sqrt(2 / (input_size + hidden_units))
        self.bias_ip = np.zeros((1, hidden_units))  # Hidden bias (1, hidden_units)
        self.weights_op = np.random.randn(hidden_units, output_size) * np.sqrt(2 / (hidden_units + output_size))
        self.bias_op = np.zeros((1, output_size))
        

    def __call__(self,X):
        return self.forward(X)

    def forward(self, X):
        self.m = X.shape[0]
        self.X = X
        self.a1 = (self.X @ self.weights_ip) + self.bias_ip
        self.z1 = relu(self.a1)
        self.a2 = (self.z1 @ self.weights_op) + self.bias_op
        self.z2 = sigmoid(self.a2)
        # print("weights= ",self.weights_ip,self.weights_op)
        # print("bias= ",self.bias_ip,self.bias_op)
        # print("a1= ",self.a1)
        # print("z1= ",self.z1)
        # print("a2= ",self.a2)
        # print("z2= ",self.z2)
        return self.z2

    def loss(self,y):
        return crossentropyloss(y,self.z2)

    def backward(self, y, learning_rate):
        dz2 = self.z2 - y        
        dw2 = (1/self.m)*(self.z1.T @ dz2)
        db2 = (1/self.m) * np.sum(dz2, axis=0, keepdims=True)
        
        da1 = dz2 @ self.weights_op.T
        
        dz1 = da1* relu_derivative(self.z1)
        
        dw1 = (1/self.m)*(self.X.T @ dz1)
        db1 = (1/self.m) * np.sum(dz1, axis=0, keepdims=True) 
        
        # print("dz2= ",dz2)
        # print("dw2=",dw2)
        # print("db2=",db2)
        # print("da1= ",da1)
        # print("dz1= ",dz1)
        # print("dw1=",dw1)
        # print("db1=",db1)
        self.weights_op = self.weights_op - learning_rate*dw2
        self.bias_op = self.bias_op - learning_rate*db2
        self.weights_ip = self.weights_ip - learning_rate*dw1
        self.bias_ip = self.bias_ip - learning_rate*db1




In [14]:
np.random.seed(42)

num_samples = 200
heights = np.random.normal(loc=170, scale=10, size=num_samples)  # Mean height ~ 170 cm
weights = np.random.normal(loc=70, scale=15, size=num_samples)    # Mean weight ~ 70 kg

bmi = weights / (heights / 100) ** 2  
y = (bmi < 24.9).astype(int)  

X = np.column_stack((heights, weights))
y = y.reshape(-1,1)


scaler = StandardScaler()
X = scaler.fit_transform(X)


model = NeuralNetwork(X.shape[1],16,1)
def train():
    epochs = 100
    for i in range(epochs):
        y_preds = model(X)
        loss = model.loss(y)
        model.backward(y,0.1)
        print(loss)

def test():
    y_preds = model(X)
    output = np.where(y_preds >= 0.5, 1, 0)
    accuracy = np.mean(output==y)
    return accuracy

In [15]:
train()

0.775194641346328
0.7450834314960167
0.7175022612296622
0.6921765042931759
0.6688657665444544
0.6471477112843749
0.6270935829680429
0.6085009288687514
0.5910594675271195
0.574613449280102
0.5590575046617438
0.544326604906801
0.5303058649793632
0.5168898200307266
0.5041351766197524
0.49197776775606483
0.4802811306436924
0.4690233637420203
0.45815878827225714
0.4476850273067417
0.4375928097190858
0.4278110193907079
0.41833058985754706
0.4091670311733761
0.40032986791623854
0.39178509416342067
0.38354195098484467
0.37553093573122565
0.3677324410995222
0.3601710612443673
0.35283793397410806
0.3457189662591251
0.3388262738084407
0.33217346081013743
0.32571929547006606
0.3194538224636058
0.31336875388012503
0.3074624278219699
0.30172461294420805
0.29616062032549
0.2907585076571587
0.28550312062952626
0.2804027587969001
0.275453508019757
0.2706391567790853
0.26597296925706776
0.26145759586425005
0.257092579071466
0.2528434501137489
0.24871962019933563
0.2447211911946873
0.24086445321338032
0.

In [16]:
test()

0.98